In [70]:
import pandas as pd
import os

Carrega a base de dados

In [ ]:
df = pd.read_csv("classificacaoCOncluida.csv")
print(df.shape)
df.head(10)


# Pre Process
    Filtrando apenas por 'Task' e suas variaçoes 
    

In [3]:
import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 



def preprocess_stop_lemma(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

def preprocess_stop(text):
    # remove stop words 
    
    doc = nlp(text)
    filtered_tokens = []
    global totaltokens
    global stopCount
   
    for token in doc:
        if token.is_stop:
            stopCount+=1
            continue
        if token.is_punct:
            continue
        totaltokens+=1
        filtered_tokens.append(token.text)   
    return " ".join(filtered_tokens)
def preprocess_lemma(text):
    # lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    global totalWords
    global noChangesCount
    for token in doc:
        if token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
        totalWords+=1
        if (str(token.lemma_)==str(token)):
            noChangesCount+=1
    return " ".join(filtered_tokens)

# Separando os dados que vamos utilizar por enquanto

Separando apenas as colunas que serao utilizadas pelo algoritimo

In [ ]:
df= df[['Name','Element_type','Category','classificationLabel']]
df

In [ ]:
df['classificationLabel'].value_counts()

Filtrando apenas por 'Task' e suas variaçoes 

In [ ]:
df_ = df[df['Element_type'].str.endswith('ask')]
df_2 = df[df['Element_type'].str.endswith('ctivity')]
df_filtred = pd.concat([df_,df_2],axis=0)
print(df_filtred.Element_type.value_counts())
df_filtred

Retirando as inconclusivas e convertendo Postivas para 1 e negativas para 0

In [7]:
df_filtred.drop(df_filtred[(df_filtred['classificationLabel'] == 'Inconclusive')].index, inplace=True)
df_filtred['Category'].value_counts()
df_filtred['bclassification'] = df_filtred['classificationLabel'].apply(lambda x: 1 if x=='Positive' else 0) 
df = df_filtred

Lower all text

In [ ]:
df['Name'] = df['Name'].str.lower()
df

## Cria dataframes para cada uma das classes adicionando colunas com os textos pre processados

Tambem é feita uma contagem de quantas palavras foram retiradas como stop words e quantas foram editadas pela lematizacao

## STT

In [ ]:
totalWords=0
noChangesCount=0
totaltokens=0
stopCount=0
df_stt = df[df['Category'].str.endswith('STT')].copy()
df_stt['Names_clean_sl'] = df_stt['Name'].apply(preprocess_stop_lemma)
df_stt['Names_clean_s'] = df_stt['Name'].apply(preprocess_stop)
print('stopCount =',stopCount,'total tokens = ',totaltokens)
df_stt['Names_clean_l'] = df_stt['Name'].apply(preprocess_lemma)
print('NoChange = ',noChangesCount,'total words =',totalWords)
print(df_stt['classificationLabel'].value_counts())
df_stt

## PP

In [ ]:
totalWords=0
noChangesCount=0
totaltokens=0
stopCount=0
df_pp = df[df['Category'].str.endswith('PP')].copy()
df_pp['Names_clean_sl'] = df_pp['Name'].apply(preprocess_stop_lemma)
df_pp['Names_clean_s'] = df_pp['Name'].apply(preprocess_stop)
print('stopCount =',stopCount,'total tokens = ',totaltokens)
df_pp['Names_clean_l'] = df_pp['Name'].apply(preprocess_lemma)
print('NoChange = ',noChangesCount,'total words =',totalWords)
print(df_pp['classificationLabel'].value_counts())
df_pp

## PDI

In [ ]:
totalWords=0
noChangesCount=0
totaltokens=0
stopCount=0
df_pdi = df[df['Category'].str.endswith('PDI')].copy()
df_pdi['Names_clean_sl'] = df_pdi['Name'].apply(preprocess_stop_lemma)
df_pdi['Names_clean_s'] = df_pdi['Name'].apply(preprocess_stop)
print('stopCount =',stopCount,'total tokens = ',totaltokens)
df_pdi['Names_clean_l'] = df_pdi['Name'].apply(preprocess_lemma)
print('NoChange = ',noChangesCount,'total words =',totalWords)
print(df_pdi['classificationLabel'].value_counts())
df_pdi

# Balanceando os datasets

## Oversampling
    para cada classe balanceamos os dados duplicando aleatoriamente elementos classificados na classe com menos elementos

In [43]:
def balance(auxdf,count,dominant):
    df_overclass = auxdf[auxdf['classificationLabel']==dominant].sample(count, replace=True, random_state=2022)
    df_False = auxdf[auxdf['classificationLabel']!=dominant]

    df_balanced = pd.concat([df_overclass, df_False],axis=0)

    return df_balanced.sample(frac=1, random_state=2022).reset_index(drop=True)
    

System, tools, and technologies (STT)

In [ ]:
x = df_stt['classificationLabel'].value_counts()
y = x.index
if x[0]!=x[1]:
    df_stt = balance(df_stt,max(x),y[1])
print(df_stt['classificationLabel'].value_counts())
df_stt

PDI

In [ ]:
x = df_pdi['classificationLabel'].value_counts()
y = x.index

if x[0]!=x[1]:
    df_pdi = balance(df_pdi,max(x),y[1])
print(df_pdi['classificationLabel'].value_counts())
df_pdi

PP

In [ ]:
x = df_pp['classificationLabel'].value_counts()
y = x.index

if x[0]!=x[1]:
    df_pp = balance(df_pp,max(x),y[1])
print(df_pp['classificationLabel'].value_counts())

df_pp

# Building the Model


In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt


In [81]:
## ecolher entre 'roc_auc' e 'accuracy'
scoretipe = 'accuracy'

Bag of words

In [60]:
def bow(dfaux,colname):
    cl1 = SVC()
    cl2 = MultinomialNB()
    cl3 = ComplementNB()
    cl4 = RandomForestClassifier()
    
    
    clf = Pipeline([('preprocessor',CountVectorizer()),('classifier',cl1)])

    params1 = {}
    params1['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params1['classifier'] = [cl1]

    params2 = {}
    params2['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params2['classifier'] = [cl2]

    params3 = {}
    params3['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params3['classifier'] = [cl3]

    params4 = {}
    params4['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params4['classifier'] = [cl4]

    params = [params1,params2,params3,params4]

    grid = GridSearchCV(clf, params ,cv=5,scoring=scoretipe)
    grid.fit(dfaux[colname],dfaux['bclassification'])


    return(colname,grid.best_params_,grid.best_score_,pd.DataFrame(grid.cv_results_)[['params','mean_test_score','rank_test_score']])



TFID

In [61]:
def tfidf(dfaux,colname):
    cl1 = SVC()
    cl2 = MultinomialNB()
    cl3 = ComplementNB()
    cl4 = RandomForestClassifier()
    
    
    clf = Pipeline([('preprocessor',TfidfVectorizer()),('classifier',cl1)])

    params1 = {}
    params1['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params1['classifier'] = [cl1]

    params2 = {}
    params2['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params2['classifier'] = [cl2]

    params3 = {}
    params3['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params3['classifier'] = [cl3]

    params4 = {}
    params4['preprocessor__ngram_range'] = [(1,1),(1,2),(2,2),(1,3),(2,3),(3,3)]
    params4['classifier'] = [cl4]

    params = [params1,params2,params3,params4]
    

    grid = GridSearchCV(clf, params ,cv=5,scoring=scoretipe)
    grid.fit(dfaux[colname],dfaux['bclassification'])
    
    return colname, grid.best_params_, grid.best_score_, pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score', 'rank_test_score']]


## Executa os testes (avaliando conforme escolhido) para todas as combinacoes de tecnicas e registra em arquivos separados

In [62]:
for x in ['Name', 'Names_clean_sl', 'Names_clean_s', 'Names_clean_l']:
    pre,params,result,df_saida = bow(df_pdi,x)
    print(f'PDI,BOW,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/PDI BOW '+pre+'.csv',index=False)

    pre,params,result,df_saida = bow(df_pdi,x)
    print(f'PDI,TF_IDF,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/PDI TF-IDF '+pre+'.csv',index=False)

    pre,params,result,df_saida = bow(df_pp,x)
    print(f'PP, BOW,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/PP BOW '+pre+'.csv',index=False)

    pre,params,result,df_saida = bow(df_pp,x)
    print(f'PP, TF_IDF,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/PP TF-IDF '+pre+'.csv',index=False)

    pre,params,result,df_saida = bow(df_stt,x)
    print(f'STT, BOW,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/STT BOW '+pre+'.csv',index=False)

    pre,params,result,df_saida = bow(df_stt,x)
    print(f'STT, TF_IDF,{pre},{params["classifier"]},{params["preprocessor__ngram_range"]},{result}')
    df_saida.to_csv(scoretipe+'/STT TF-IDF '+pre+'.csv',index=False)



PDI,BOW,Name,MultinomialNB(),(1, 2),0.9237500000000001
PDI,TF_IDF,Name,MultinomialNB(),(1, 2),0.9237500000000001
PP, BOW,Name,SVC(),(1, 1),0.8882807017543859
PP, TF_IDF,Name,SVC(),(1, 1),0.8882807017543859
STT, BOW,Name,RandomForestClassifier(),(1, 3),0.9417176372015081
STT, TF_IDF,Name,RandomForestClassifier(),(1, 3),0.9417176372015081
PDI,BOW,Names_clean_sl,MultinomialNB(),(1, 2),0.92875
PDI,TF_IDF,Names_clean_sl,MultinomialNB(),(1, 2),0.92875
PP, BOW,Names_clean_sl,RandomForestClassifier(),(1, 1),0.8989473684210527
PP, TF_IDF,Names_clean_sl,RandomForestClassifier(),(1, 1),0.8936491228070175
STT, BOW,Names_clean_sl,RandomForestClassifier(),(2, 2),0.9520821114369502
STT, TF_IDF,Names_clean_sl,RandomForestClassifier(),(2, 3),0.952073732718894
PDI,BOW,Names_clean_s,MultinomialNB(),(1, 2),0.9299999999999999
PDI,TF_IDF,Names_clean_s,MultinomialNB(),(1, 2),0.9299999999999999
PP, BOW,Names_clean_s,RandomForestClassifier(),(1, 1),0.8963508771929825
PP, TF_IDF,Names_clean_s,RandomForestClassi

## Concatenando as diferentes saidas em um unico arquivo para roc_auc e Accuracy

In [74]:
accuracy_list = os.listdir('accuracy')
roc_auc_list = os.listdir('roc_auc')

In [79]:
def agrupar_saidas(lista,pasta):
    colunas = ['class','algorithm','textPreprocess']
    df_completo = pd.DataFrame()
    for arquivo in lista:
        df_parcial = pd.read_csv(pasta+'/'+arquivo)
        arquivo = arquivo.replace('.csv',' ')
        padroes_do_arquivo = arquivo.split()
        for ind,col in enumerate(padroes_do_arquivo):
            df_parcial[colunas[ind]] = col
        df_completo = pd.concat([df_completo,df_parcial],axis=0)

    df_completo.to_csv(f'results_{pasta}_completo.csv',index=False)

In [80]:
agrupar_saidas(accuracy_list,"accuracy")
agrupar_saidas(roc_auc_list,"roc_auc")

## Executa novo treino com os melhores casos de n-gramas em cada combinaçao classe - algoritimo para gerar grafico

In [82]:
def split(names_column,df_class):
    X_train, X_test, y_train, y_test = train_test_split(
        df_class[names_column],
        df_class['bclassification'],
        test_size=0.2, # 20% samples will go to test dataset
        stratify=df_class['bclassification'],
        random_state=1873211
        )
    return X_train, X_test, y_train, y_test

In [83]:
def tfidf2(X_train, X_test, y_train, y_test,classe,best):

    models = [MultinomialNB(),ComplementNB(),RandomForestClassifier(),SVC()]
    names = ['MultinomialNB','ComplementNB','RFC','SVC']
    for name,mod in enumerate(models):
        v = TfidfVectorizer(ngram_range=best[str(mod)])
        X_train_count = v.fit_transform(X_train.values)
        model = mod
        model.fit(X_train_count,y_train)


        X_test_count = v.transform( X_test)
        svc_disp = RocCurveDisplay.from_estimator(model, X_test_count, y_test, ax=ax,name=classe + names[name] +str(best[str(mod)]))


In [ ]:
df_stt.name='STT '
df_pp.name='PP '
df_pdi.name='PDI '
dfs = [df_stt,df_pp,df_pdi]

#selecionar o numero de ngramas que apresentaram melhor desempenho para cada algoritimo em cada classe
bestpdi= {'MultinomialNB()':(1,2),'ComplementNB()':(1,2),'RandomForestClassifier()':(1,1),'SVC()':(1,1)}
bestpp = {'MultinomialNB()':(1,2),'ComplementNB()':(2,2),'RandomForestClassifier()':(1,1),'SVC()':(1,1)}
beststt = {'MultinomialNB()':(1,3),'ComplementNB()':(2,3),'RandomForestClassifier()':(2,2),'SVC()':(1,3)}
bests = [beststt,bestpp,bestpdi]
plt.figure(figsize=[5.5,4],frameon=False)

ax = plt.gca()
ax.set_facecolor("white")



for x,df_ in enumerate(dfs):
    X_train, X_test, y_train, y_test = split('Names_clean_sl',df_)
    tfidf2(X_train, X_test, y_train, y_test,df_.name,bests[x])
plt.grid(color='#ececec')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(facecolor="white")
ax.spines['bottom'].set_color('#4d4d4d')
ax.spines['top'].set_color('#4d4d4d')
ax.xaxis.label.set_color('#4d4d4d')
ax.tick_params(axis='x', colors='#4d4d4d')

ax.spines['left'].set_color('#4d4d4d')
ax.spines['right'].set_color('#4d4d4d')
ax.xaxis.label.set_color('#4d4d4d')
ax.tick_params(axis='y', colors='#4d4d4d')

plt.savefig('line_plot.pdf')